## Predicting diseases  - Aggregation vs Mitochondrial toxicity with PowerTransformer


In [ ]:
%load_ext autoreload
%autoreload 2

# Imports

In [ ]:
import pandas as pd
import numpy as np

from pathlib import Path 

import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
from pprint import pprint

In [ ]:
from IPython.display import display_html

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
from scipy.stats import gaussian_kde

from numpy import  interp

from itertools import cycle

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report, roc_auc_score, roc_curve, auc

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
from tensorflow.random import set_seed

In [ ]:
set_seed(42)

In [ ]:
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model


In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
import pickle

import shap

In [ ]:
pd.set_option("display.max_columns", 100)  

In [ ]:
from sklearn.preprocessing import PowerTransformer
from scipy import optimize
class SafePowerTransformer(PowerTransformer):
    def _yeo_johnson_optimize(self, x):
        def _neg_log_likelihood(lmbda):
            x_trans = self._yeo_johnson_transform(x, lmbda)
            n_samples = x.shape[0]
            variance = x_trans.var()
            if variance == 0:
                return np.inf  
            loglike = -n_samples / 2 * np.log(variance) 
            loglike += (lmbda - 1) * (np.sign(x) * np.log1p(np.abs(x))).sum()
            return -loglike
        x = x[~np.isnan(x)]
        return optimize.brent(_neg_log_likelihood, brack=(-2, 2))

In [ ]:
img_dpi= 300

# My package

In [ ]:
from neurocrick import plotter
from neurocrick.utils import display_side_by_side

In [ ]:
from sinfo import sinfo
sinfo()

# Set paths

## Read data

In [ ]:
data_file = Path(os.path.abspath("../demo_data/coloc_tab_demo_data.csv"))

## Results

In [ ]:
path_res = Path(os.path.abspath("../demo_output/coloc_tab_data_aggr_mt/"))
print(path_res)
path_res.is_dir()

In [ ]:
path_plots = path_res.joinpath("plots")
print(path_plots)
path_plots.is_dir()

In [ ]:
path_models = path_res.joinpath("saved_models")
print(path_models)
path_models.is_dir()

In [ ]:
if not path_res.is_dir():
    path_res.mkdir(parents=True, exist_ok=True)

if not path_plots.is_dir():
    path_plots.mkdir(parents=True, exist_ok=True)
    
if not path_models.exists():
    path_models.mkdir(parents=True, exist_ok=True)

## Load data

In [ ]:
df = pd.read_csv(data_file, index_col=0)
print(df.shape)
display(df.head(3))

In [ ]:
df.label_name.value_counts()

In [ ]:
list(df.columns)

In [ ]:
df = df.drop(columns=['Unnamed: 1'])

In [ ]:
# lysosome morphology length, width and ratio of width to length is there
col_exclude= ['lysosome morphology Ratio Width to Length']

In [ ]:
df.shape

In [ ]:
df = df.drop(col_exclude, axis=1)
df.shape

In [ ]:
df=df.reset_index()
df = df.drop(columns=['index'])
df.head(4)

In [ ]:
experiment_cols = ["PlateID"]

In [ ]:
print( len(df.columns))
all_feats = [col for col in df.columns if 'label' not in col and 'PlateID' not in col]
len(all_feats)

# Explore data

In [ ]:
df[df.duplicated()]

There are no duplicates data

In [ ]:
df.label.value_counts()

In [ ]:
df.label_name.value_counts()

In [ ]:
df.loc[(df.label_name=='Oligomycin') | (df.label_name=='rotenone'), "label_name"] = 'mito_toxicity'
df.loc[(df.label_name=='3xSNCA') | (df.label_name=='oligomer'), "label_name"] = 'aggregation'

In [ ]:
df.label_name.value_counts()

In [ ]:
#indicates disease
df.loc[df['label_name']=="aggregation","label"] = 0 
df.loc[df['label_name']=="mito_toxicity","label"] = 1
df.loc[df['label_name']=="control","label"] = 2


In [ ]:
label_dict = {
    0: "Aggregation",
    1: "Mitochondrial toxicity",
    2: "control"
}
label_dict[0]

In [ ]:
#Total n of controls and diseases 
display_side_by_side(
    pd.DataFrame(df.label_name.value_counts()).rename(columns={"label_name":'label_count'}), 
    pd.DataFrame(np.round(df.label_name.value_counts()/len(df)*100, decimals=2)).rename(columns={"label_name":'percent'})
)

In [ ]:
df["PlateID"] = df.PlateID.astype('category')
df["label_name"] = df.label_name.astype('category')

In [ ]:
label_cols = ['label', 'label_name']

#### How many plates

In [ ]:
display_side_by_side(
    pd.DataFrame(df.PlateID.value_counts()).rename(columns={"PlateID":'PlateID_count'}), 
    pd.DataFrame(np.round(df.PlateID.value_counts()/len(df)*100, decimals=2)).rename(columns={"PlateID":'PlateID_percent'})
)

#### How many labels per plate

In [ ]:
df_overview = pd.concat([
    pd.DataFrame(
        df.groupby(["PlateID", 'label_name'])["label"].count()
    ).rename(columns={"label":'count_label'}),
    pd.DataFrame(
        np.round(
            df.groupby(["PlateID", 'label_name'])["label"].count()/len(df)*100, decimals=1
        )
    ).rename(columns={"label":'percent_of_dataset'}),
    pd.DataFrame(
        np.round(
            pd.DataFrame(
                df.groupby(["PlateID", 'label_name']).agg({'label': 'count'})
            ).groupby(level=0).apply(lambda x: 100 * x / float(x.sum())),
            decimals=1
        )
    ).rename(columns={"label":'percent_of_plate'}),
], axis=1)

df_overview


#### How many nans and where overall

In [ ]:
mask = df.isnull().sum()
mask2 = df.isnull().sum()/len(df)*100

pd.concat([
    pd.DataFrame(mask[mask>0], columns=["total_missing"]), 
    np.round(pd.DataFrame(mask2[mask2>0], columns=["percent_missing"]), decimals=2)],
    axis=1
)

In [ ]:
df["PlateID"] = df.PlateID.astype('category')

#### Split into train, val and test datasets 


In [ ]:
display(df.head())
df= df.sample(frac=1, random_state=200)
display(df.head())

In [ ]:
#train_test_split
df_train, df_val = train_test_split(df,test_size=0.2, random_state=42)

df_train, df_test = train_test_split(df_train, test_size=0.2, random_state=42)
    

In [ ]:
display(df_train.shape)
display(df_val.shape)
display(df_test.shape)


#### CHECK DISTRIBUTIONS OF DATA per plate per label in the training set


ctrl vs aggregation vs mito_toxicity   

In [ ]:
len(all_feats)

In [ ]:
experiment_cols+all_feats

In [ ]:
label_cols

In [ ]:
df_train.shape

##### Number of control, disease points per plate in train and val set

In [ ]:
def display_counts_pct_per_plate(df):
    df_overview = pd.concat([
        pd.DataFrame(
            df.groupby(["PlateID", 'label_name'])["label_name"].count()
        ).rename(columns={"label_name":'count_label'}),
        pd.DataFrame(
            np.round(
                df.groupby(["PlateID", 'label_name'])["label_name"].count()/len(df)*100, decimals=1
            )
        ).rename(columns={"label_name":'percent_of_dataset'}),
        pd.DataFrame(
            np.round(
                pd.DataFrame(
                    df.groupby(["PlateID", 'label_name']).agg({'label_name': 'count'})
                ).groupby(level=0).apply(lambda x: 100 * x / float(x.sum())),
                decimals=1
            )
        ).rename(columns={"label_name":'percent_of_plate'}),
    ], axis=1)

    return(df_overview)

In [ ]:
display_counts_pct_per_plate(df_train)

In [ ]:
display_counts_pct_per_plate(df_val)

In [ ]:
display_side_by_side(
    pd.DataFrame(df_train.label_name.value_counts()).rename(columns={"label_name":'label_count'}), 
    pd.DataFrame(np.round(df_train.label_name.value_counts()/len(df_train)*100, decimals=2)).rename(columns={"label_name":'percent'})
)

In [ ]:
display_side_by_side(
    pd.DataFrame(df_val.label_name.value_counts()).rename(columns={"label_name":'label_count'}), 
    pd.DataFrame(np.round(df_val.label_name.value_counts()/len(df_val)*100, decimals=2)).rename(columns={"label_name":'percent'})
)

# Make model aggregation vs mito_toxicity
### Powerscale PER CONTROL, WITHIN EACH PLATE SEPARATELY (ie. per control per plate)

### Fill the nans in each plate according to the other feature values IN THAT PLATE

### Use a power transformer to scale the features within each plate


In [ ]:
print(df_train[all_feats].isnull().sum().sum())
df_train[all_feats].isnull().sum().sum()/len(df)*100.

In [ ]:
def scale_trans_by_control_per_plate(df, feat_cols):
    imp = IterativeImputer(max_iter=10, random_state=0, initial_strategy='median')
    
    lambdas = {}

    fin_df = []
    for eachplate in df.PlateID.unique():
        dd = df[df.PlateID==eachplate].copy()

        print(f"plate:{eachplate}, shape:{dd.shape}")

        # fit for scaling. Scale per control per plate, then apply the transformation to all datapoints in the plate.
        scaler = SafePowerTransformer()
        #scaler.fit(dd[dd.label==0][all_feats].values)
        scaler.fit(dd[dd.label_name=='control'][feat_cols].values)

        # scale
        dd.loc[:,feat_cols] = scaler.transform(dd[feat_cols].values)

        # inpute missing values using iterative inputer
        dd.loc[:,feat_cols] = imp.fit_transform(dd[feat_cols].values)

        lambdas[eachplate] = scaler.lambdas_
        fin_df.append(dd)

    fin_df = pd.concat(fin_df)
    #print(fin_df.shape)
    return fin_df, lambdas


In [ ]:
scal_df_train, lambdas_train = scale_trans_by_control_per_plate(df_train, all_feats)
print(scal_df_train.shape)
scal_df_train.head()

In [ ]:
print(df_train.label.value_counts(), "\n", df_train.label_name.value_counts())
print("After SCALED")
print(scal_df_train.label.value_counts(), "\n", scal_df_train.label_name.value_counts())


In [ ]:
scal_df_train = scal_df_train.sample(frac = 1, random_state=200) #shuffle
scal_df_train.head(5)

#### Check the scaling

In [ ]:
with np.printoptions(precision=2, suppress=True):
    print(np.array(list(lambdas_train.values())))

lambdas_train_df = pd.DataFrame(np.array(list(lambdas_train.values())), columns=all_feats, index=list(lambdas_train.keys()))
lambdas_train_df.index.rename("Plate_ID", inplace=True)
lambdas_train_df

lambdas_train_df.shape
title = "Scaling factor for CONTROL distribution across plates for the training data" 
fig, ax = plt.subplots(figsize=(8,12))
sns.pointplot(
    data=lambdas_train_df, 
    join=False, 
    orient='h',
    order=list(lambdas_train_df.std().sort_values(ascending=False).index),
)

ax.set_xlabel("Average value of scaling factor (with std error) for CONTROL distribution across plates ")
fig.savefig( path_plots.joinpath(f"scale_plate-control-powertrans_{title}.png"), bbox_inches='tight', dpi= img_dpi)

##### Checking the variance of the lambdas

In [ ]:

lambdas_var = lambdas_train_df.var().sort_values(ascending = False)
lambdas_var

##### Check it looks scaled

In [ ]:
def plot_violin_4_features(df, var_types):
    # path_plots.joinpath('distributions_condition-grouped_scaled').mkdir(exist_ok=True)

    for var_type in var_types[1:]:
        title = var_type

        g = sns.PairGrid(df,
                         x_vars=var_type, 
                         hue='label_name',
                         y_vars=["PlateID"],
                         hue_order=['control', 'aggregation', 'mito_toxicity'],
                         height=8, aspect=0.7)

        # Draw a dot plot using the stripplot function
        g.map(sns.violinplot, inner="quartile", zorder=1, dodge=0.2)

        for ax in g.axes.flat:
            for violin in ax.collections:
                violin.set_alpha(.8)

            for l in ax.lines:
                l.set_linestyle('--')
                l.set_linewidth(0.6)
                l.set_color('white')
                l.set_alpha(0.8)
            for l in ax.lines[1::3]:
                l.set_linestyle('-')
                l.set_linewidth(1.3)
                l.set_color('black')
                l.set_alpha(0.8)

            for i in range(4):
                ax.axhline(i + 0.5, color='grey', lw=0.7)

            ax.vlines(x=-1, ymin=-0.5, ymax=4.8, lw=0.5)
            ax.vlines(x=1, ymin=-0.5, ymax=4.8, lw=0.5)

        ax = g.axes.flat[0]
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(
            handles, labels,
            handletextpad=0.5, columnspacing=1,
            bbox_to_anchor=(0, 1.1), loc='upper left',
            fontsize=13,
            ncol=3,
            frameon=False,
        )

        fig = plt.gcf()
        fig.suptitle(title, fontsize=15, y=1.1)
        #fig.savefig(path_plots.joinpath('distributions_condition-grouped_scaled').joinpath(f'{title}.png'), bbox_inches='tight')
        plt.show()
    return(True)

In [ ]:
plot_violin_4_features(scal_df_train, np.asarray(all_feats))

In [ ]:
scal_df_train.head(2)

### Correlations amongs features and labels

In [ ]:
th = 0.3
f = plotter.make_corr_matrix_plot(
    scal_df_train.drop(columns=['label_name']+experiment_cols),
    mask_less_than=th, fmt= '.2f',
);

f.tight_layout()
f.suptitle(f"Correlation matrix, all plates and conditions above {th} correlation", fontsize=22)
# f.savefig(path_plots.joinpath(f"corr-matrix-above{th}_diseases-control_powertransfor-control-plate.png"), bbox_inches='tight')

#### Exclude the features with a high variance in the lambdas in the training set

In [ ]:
var_cutoff = 50 # to exclude the single one with the max variance


In [ ]:
col_2_exclude = lambdas_var[lambdas_var > var_cutoff].index
col_2_exclude

In [ ]:
scal_df_train.shape

In [ ]:
#exclude the features with high variance
scal_df_train = scal_df_train.drop(col_2_exclude, axis=1)
scal_df_train.shape

In [ ]:
scal_df_train.head(2)

In [ ]:
th = 0.3
f = plotter.make_corr_matrix_plot(
    scal_df_train.drop(columns=['label_name']+experiment_cols),
    mask_less_than=th, fmt= '.2f',
);

f.tight_layout()
f.suptitle("Correlation matrix, all plates and conditions above {th} correlation", fontsize=22)
# f.savefig(path_plots.joinpath(f"corr-matrix-above{th}_diseases-control_powertransfor-control-plate.png"), bbox_inches='tight')

#### Scaling the test and validation set and exclude the same features as in the training set 

##### Scale and transform the val and test set
Powerscale PER CONTROL, WITHIN EACH PLATE SEPARATELY (ie. per control per plate)

In [ ]:
#scaling
scal_df_val, lambdas_val = scale_trans_by_control_per_plate(df_val, all_feats)
print(scal_df_val.shape)
scal_df_val.head()

In [ ]:
scal_df_val = scal_df_val.sample(frac= 1, random_state = 200)

In [ ]:
#scaling
scal_df_test, lambdas_test = scale_trans_by_control_per_plate(df_test, all_feats)
print(scal_df_test.shape)
scal_df_test = scal_df_test.sample(frac= 1, random_state = 200)
scal_df_test.head(5)

In [ ]:
lambdas_val_df = pd.DataFrame(np.array(list(lambdas_val.values())), columns=all_feats, index=list(lambdas_val.keys()))
lambdas_val_df.index.rename("Plate_ID", inplace=True)
lambdas_val_df

In [ ]:
fig, ax = plt.subplots(figsize=(8,12))
sns.pointplot(
    data=lambdas_val_df, 
    join=False, 
    orient='h',
    order=list(lambdas_val_df.std().sort_values(ascending=False).index),
)

ax.set_xlabel("Val data- Average value of scaling factor (with std error) for CONTROL distribution across plates ")

In [ ]:
#Checkign the var of the lambdas

lambdas_var_val = lambdas_val_df.var().sort_values(ascending = False)
lambdas_var_val

##### Checking the features in val and training dataset,  with variance < the cutoff

In [ ]:
print(f"No. of feature in the training set that pass the lambdas variance threshold of {var_cutoff}  = {len( lambdas_var[lambdas_var < var_cutoff])}")
print(f"\nNo. of features in the val set that pass the threshold = {len( lambdas_var_val[lambdas_var_val < var_cutoff])}")

print(f"\nNo. of features that overlap between the 2 = {len( lambdas_var_val[lambdas_var_val<var_cutoff].index.intersection( lambdas_var[lambdas_var < var_cutoff].index ))}")

##### Dropping the same features from the val and test set as in the training set

In [ ]:
scal_df_val.shape

In [ ]:
scal_df_val = scal_df_val.drop(col_2_exclude, axis=1)
scal_df_val.shape

In [ ]:
scal_df_test.shape

In [ ]:
scal_df_test = scal_df_test.drop(col_2_exclude, axis=1)
scal_df_test.shape

In [ ]:
scal_df_train.shape

In [ ]:
screen_feats = list(set(all_feats)-set(col_2_exclude))
screen_feats

### Remove control from the test, train and val datasets

In [ ]:
scal_df_train.head()

In [ ]:
#Dropping the control as looking at diseases
def drop_observations_control(df, control_code):
    filter_dfs = df[df.label!=control_code].copy()

    filter_dfs['label'] = filter_dfs.label - min(filter_dfs.label.unique())

    filter_dfs.label_name = filter_dfs.label_name.cat.remove_unused_categories()

    filter_dfs = filter_dfs.reset_index(drop=True)
    
    return filter_dfs

In [ ]:
control_code = (scal_df_train.loc[scal_df_train.label_name =="control"].label.unique())[0]
print(control_code)


In [ ]:
print("Train data : Before dropping the controls")
print(scal_df_train.shape, "\n", scal_df_train.label_name.value_counts() )

scal_df_train = drop_observations_control(scal_df_train, control_code)
print("\nAfter dropping the controls")

print(scal_df_train.shape)
scal_df_train.label_name.value_counts()


In [ ]:
print("Val data : Before dropping the controls")
print(scal_df_val.shape, "\n", scal_df_val.label_name.value_counts() )

scal_df_val = drop_observations_control(scal_df_val, control_code)
print("\nAfter dropping the controls")
print(scal_df_val.shape)
scal_df_val.label_name.value_counts()


In [ ]:
print("Test data : Before dropping the controls")
print(scal_df_test.shape, "\n", scal_df_test.label_name.value_counts() )

scal_df_test = drop_observations_control(scal_df_test, control_code)
print("\nAfter dropping the controls")
print(scal_df_test.shape)
scal_df_test.label_name.value_counts()


In [ ]:
screen_feats

In [ ]:
#Saving the train, val and test data
scal_df_train.to_csv(path_res.joinpath('scal_df_train.csv'),index=False)
scal_df_val.to_csv(path_res.joinpath('scal_df_val.csv'),index=False)
scal_df_test.to_csv(path_res.joinpath('scal_df_test.csv'),index=False)

#Saving the screen feats
with open(path_res.joinpath('screen_feats.csv'), 'w') as FOUT:
    np.savetxt(FOUT, screen_feats, delimiter =",",fmt ='% s')


In [ ]:
x_train = scal_df_train[screen_feats].values
x_val = scal_df_val[screen_feats].values
x_test = scal_df_test[screen_feats].values

y_train = scal_df_train['label'].values
y_val = scal_df_val['label'].values
y_test = scal_df_test['label'].values


In [ ]:
y_train.shape

In [ ]:
print(y_val.shape)
y_test.shape

In [ ]:
x_train.shape

In [ ]:
print(x_val.shape)
x_test.shape

### ML: Dense Neural Network 

In [ ]:
y_test_onehot = to_categorical(y_test)
y_test_onehot

In [ ]:
y_val_onehot = to_categorical(y_val)
y_val_onehot

In [ ]:
y_train_onehot = to_categorical(y_train)
y_train_onehot

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=0, 
    patience=50, 
    verbose=0, 
    mode='auto',
    baseline=None, 
    restore_best_weights=True,
)

In [ ]:
num_dense = 25 #40
scal_model_name =  path_models.joinpath(f"mixed-plates_coloc_aggr_mitoTox_lambThresh{var_cutoff}_pTrans-control-plate-scaling_dense-{num_dense}_3-dropout")
scal_model_name

In [ ]:
scal_checkpoint = ModelCheckpoint(
    str(scal_model_name)+'.hdf5', 
    monitor='val_loss', 
    verbose=1,
    save_best_only=True, 
    mode='auto', 
    save_freq='epoch'
)

In [ ]:
scal_model_descr = ("Coloc - Predicting diseases across plates, "
                    f"Power transform per control within plate,\n model: {os.path.basename(scal_model_name)}")
scal_model_descr

In [ ]:
dropout_rate = 0.2

input_t = Input(shape=(x_train.shape[1],))

x = Dense(num_dense, activation='relu')(input_t)
x = Dropout(dropout_rate)(x)

x = Dense(num_dense, activation='relu')(x)
x = Dropout(dropout_rate)(x)

x = Dense(num_dense, activation='relu')(x)
x = Dropout(dropout_rate)(x)


output = Dense(y_train_onehot.shape[1], activation='softmax')(x)

scal_model = Model(inputs=input_t, outputs = output)

In [ ]:
scal_model.summary()

In [ ]:
scal_model.compile(
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    optimizer='adam',
)

In [ ]:
batch_size = 64
epochs = 200 
history = scal_model.fit(
    x_train,
    y_train_onehot,
    validation_data = (x_val, y_val_onehot),
    batch_size = batch_size,
    epochs = epochs,
    callbacks=[scal_checkpoint, early_stop],
)

#### Plot model results

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(14,7))

ax1.plot(history.history['loss'], label='Training')
ax1.plot(history.history['val_loss'], label='Validation')
ax1.legend(frameon=False)
ax1.set_title("Loss")
ax1.set_ylabel("Loss")
ax1.set_xlabel("Epochs")

ax2.plot(history.history['accuracy'], label='Training')
ax2.plot(history.history['val_accuracy'], label='Validation')
ax2.legend(frameon=False)
ax2.set_title("Accuracy")
ax2.set_ylabel("Accuracy")
ax2.set_xlabel("Epochs")

fig.suptitle(scal_model_descr);
fig.savefig(path_plots.joinpath(f"Model-perform_{os.path.basename(scal_model_name)}_Scale-controlplate-powertrans.png"), bbox_inches='tight', dpi = img_dpi)


In [ ]:
scal_model_saved = tf.keras.models.load_model(str(scal_model_name)+'.hdf5')

In [ ]:
y_pred = scal_model_saved.predict(x_test)

In [ ]:
y_pred

In [ ]:
[str(scal_df_train.label_name.cat.categories[i]) for i in range(scal_df_train.label_name.nunique())]


In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(14,7))

ax1.set_title("Count")
scal_matrix = confusion_matrix(
    y_test_onehot.argmax(axis=1), y_pred.argmax(axis=1)
)

scal_matrix_norm = confusion_matrix(
    y_test_onehot.argmax(axis=1), y_pred.argmax(axis=1),
    normalize='true',
)

ConfusionMatrixDisplay(
    confusion_matrix=scal_matrix,
    display_labels=[str(label_dict[i]) for i in range(scal_df_train.label_name.nunique())]
).plot(cmap='Blues', ax=ax1, values_format="g")

ax1.set_ylabel(ax1.get_ylabel(), fontsize=15)
ax1.set_xlabel(ax1.get_xlabel(), fontsize=15, labelpad=10)
fig.axes[-1].set_visible(False)

# Saving the counts and fraction separately using extent
extent = ax1.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
# Pad the saved area by 70% in the x-direction and 20% in the y-direction
fig.savefig(path_plots.joinpath(f"CONFMATRIX_count_exp_{os.path.basename(scal_model_name)}_Scale-controlplate.png"),
            bbox_inches=extent.expanded(1.85, 1.2), dpi=img_dpi )

ax2.set_title("Fraction")
ConfusionMatrixDisplay(
    confusion_matrix=scal_matrix_norm,
    display_labels=[str(label_dict[i]) for i in range(scal_df_train.label_name.nunique())]
).plot(cmap='Blues', ax=ax2)

ax2.set_ylabel(ax2.get_ylabel(), fontsize=15)
ax2.set_xlabel(ax2.get_xlabel(), fontsize=15, labelpad=10)
fig.axes[-1].set_visible(False)

extent = ax2.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
# Pad the saved area in the x-direction and y-direction
fig.savefig(path_plots.joinpath(f"CONFMATRIX_fraction_exp_{os.path.basename(scal_model_name)}_Scale-controlplate.png"),
            bbox_inches=extent.expanded(1.85, 1.2), dpi=img_dpi)

#fig.tight_layout()
fig.suptitle(f"CONFUSION MATRIX\n{scal_model_descr}", y=1.01)
fig.savefig(path_plots.joinpath(f"CONFMATRIX_{os.path.basename(scal_model_name)}_Scale-controlplate.png"), bbox_inches='tight', dpi = img_dpi)
plt.show()


print('\n\n', 
      classification_report(y_test_onehot.argmax(axis=1), y_pred.argmax(axis=1))
     )


In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred[:, 1])
auc_score =  roc_auc_score(y_test, y_pred[:, 1])
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(
    fpr,
    tpr,
    color='darkorange',
    lw=2, 
    label='ROC curve (area = %0.2f)' % auc_score
)
ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title(f'{scal_model_descr}')
ax.legend(loc="lower right", frameon=False)
fig.suptitle('ROC Curve', fontsize=15, y=1.01)
fig.savefig(path_plots.joinpath(f"ROC_{os.path.basename(scal_model_name)}_Scale-controlplate.png"), 
            bbox_inches='tight', dpi= img_dpi)
fig.show()